In [8]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
!unzip "/content/drive/My Drive/Data/4_class_data.zip" -d "/content/data"

Archive:  /content/drive/My Drive/Data/4_class_data.zip
   creating: /content/data/data/
   creating: /content/data/data/test/
   creating: /content/data/data/test/backpack/
  inflating: /content/data/data/test/backpack/00000010.png  
  inflating: /content/data/data/test/backpack/00000018.png  
  inflating: /content/data/data/test/backpack/00000027.png  
  inflating: /content/data/data/test/backpack/00000053.png  
  inflating: /content/data/data/test/backpack/00000069.png  
  inflating: /content/data/data/test/backpack/00000131.png  
 extracting: /content/data/data/test/backpack/00000137.png  
  inflating: /content/data/data/test/backpack/00000199.png  
  inflating: /content/data/data/test/backpack/00000214.png  
  inflating: /content/data/data/test/backpack/00000217.png  
  inflating: /content/data/data/test/backpack/00000245.png  
  inflating: /content/data/data/test/backpack/00000269.png  
  inflating: /content/data/data/test/backpack/00000270.png  
  inflating: /content/data/data/t

In [4]:
IMAGE_SIZE = [224, 224]

In [20]:
train_path = '/content/data/data/train'
test_path = '/content/data/data/test'

In [10]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 3s 0us/step


In [11]:
for layer in vgg.layers:
  layer.trainable = False

In [19]:
folders = glob('/content/data/data/train/*')
print(len(folders))

4


In [21]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [22]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [23]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [24]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [25]:
train_set = train_datagen.flow_from_directory('/content/data/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 463 images belonging to 4 classes.


In [26]:
test_set = test_datagen.flow_from_directory('/content/data/data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 256 images belonging to 4 classes.


In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

checkpoint = ModelCheckpoint(filepath='vgg.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=15,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/15


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


5/5 [==============================] - 431s 86s/step - loss: 5.2056 - accuracy: 0.6375 - val_loss: 6.7100 - val_accuracy: 0.6172

Epoch 00001: val_loss improved from inf to 6.70998, saving model to vgg.h5
Epoch 2/15
5/5 [==============================] - 412s 82s/step - loss: 6.0075 - accuracy: 0.6094 - val_loss: 6.2307 - val_accuracy: 0.6172

Epoch 00002: val_loss improved from 6.70998 to 6.23069, saving model to vgg.h5
Epoch 3/15
5/5 [==============================] - 419s 84s/step - loss: 5.5965 - accuracy: 0.6364 - val_loss: 5.2721 - val_accuracy: 0.6172

Epoch 00003: val_loss improved from 6.23069 to 5.27213, saving model to vgg.h5
Epoch 4/15
5/5 [==============================] - 412s 82s/step - loss: 5.3827 - accuracy: 0.6500 - val_loss: 5.7514 - val_accuracy: 0.6172

Epoch 00004: val_loss did not improve from 5.27213
Epoch 5/15
5/5 [==============================] - 413s 83s/step - loss: 6.0075 - accuracy: 0.6094 - val_loss: 5.2721 - val_accuracy: 0.6172

Epoch 00005: val_loss 